## Using the agents

In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline, TextStreamer
import json
import torch
import textwrap
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings

############# DEFINE SOME BASE PARSING AND PROMPTING WRAPPERS
import json
import textwrap

load_in_8bit=QUANTIZATION_USE_8BIT = True
load_in_4bit=QUANTIZATION_USE_4BIT = False
USE_CACHE = False

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS if new_system_prompt is not None else ""
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_print_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text
    

model_name = "meta-llama/Llama-2-13b-chat-hf" # "meta-llama/Llama-2-13b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=True,)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=True,
                                            load_in_8bit=QUANTIZATION_USE_8BIT,
                                            load_in_4bit=QUANTIZATION_USE_4BIT,
                                             )

streamer = TextStreamer(tokenizer, skip_prompt=True)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens=4096,
                do_sample=True,
                temperature=0.1,
                top_p=0.95,
                min_length=None, #The minimum length of the sequence to be generated, input prompt + min_new_tokens,
                repetition_penalty=1.0, #The parameter for repetition penalty. 1.0 means no penalty.
                length_penalty=1, #[optional] Exponential penalty to the length that is used with beam-based generation.              
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                streamer=streamer)

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.9})



KeyboardInterrupt



In [2]:
llm(get_prompt("Hello. How are you?", ""))#DEFAULT_SYSTEM_PROMPT))

/home/ciprian/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Hello! I'm just an AI, I don't have feelings or emotions, so I can't say how I am. However, I'm here to help answer any questions you may have to the best of my ability. How can I assist you today?</s>


"  Hello! I'm just an AI, I don't have feelings or emotions, so I can't say how I am. However, I'm here to help answer any questions you may have to the best of my ability. How can I assist you today?"

In [ ]:
import re
from statistics import mode

def gen_answer():
    response = llm(
        "John found that the average of 15 numbers is 40."
        "If 10 is added to each number then the mean of the numbers is?"
        "Report the answer surrounded by three backticks, for example: ```123```",
        model = DEFAULT_MODEL, #LLAMA2_70B_CHAT
    )
    match = re.search(r'```(\d+)```', response)
    if match is None:
        return None
    return match.group(1)

answers = [gen_answer() for i in range(5)]

print(
    f"Answers: {answers}\n",
    f"Final answer: {mode(answers)}",
    )

import re
from statistics import mode

def gen_answer():
    response = completion(
        "John found that the average of 15 numbers is 40."
        "If 10 is added to each number then the mean of the numbers is?"
        "Report the answer surrounded by three backticks, for example: ```123```",
        model = DEFAULT_MODEL, #LLAMA2_70B_CHAT
    )
    match = re.search(r'```(\d+)```', response)
    if match is None:
        return None
    return match.group(1)

answers = [gen_answer() for i in range(5)]

print(
    f"Answers: {answers}\n",
    f"Final answer: {mode(answers)}",
    )

complete_and_print(
    """
    # Python code to calculate: ((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
    """,
    model="meta/codellama-34b:67942fd0f55b66da802218a19a8f0e1d73095473674061a6ea19f2dc8c053152"
)

complete_and_print("""
Calculate the answer to the following math problem:

((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
""")

MENLO_PARK_TEMPS = {
    "2023-12-11": "52 degrees Fahrenheit",
    "2023-12-12": "51 degrees Fahrenheit",
    "2023-12-13": "51 degrees Fahrenheit",
}


def prompt_with_rag(retrived_info, question):
    complete_and_print(
        f"Given the following information: '{retrived_info}', respond to: '{question}'"
    )


def ask_for_temperature(day):
    temp_on_day = MENLO_PARK_TEMPS.get(day) or "unknown temperature"
    prompt_with_rag(
        f"The temperature in Menlo Park was {temp_on_day} on {day}'",  # Retrieved fact
        f"What is the temperature in Menlo Park on {day}?",  # User question
    )


ask_for_temperature("2023-12-12")
# "Sure! The temperature in Menlo Park on 2023-12-12 was 51 degrees Fahrenheit."

ask_for_temperature("2023-07-18")


### Limiting Extraneous Tokens

A common struggle is getting output without extraneous tokens (ex. "Sure! Here's more information on...").

Check out this improvement that combines a role, rules and restrictions, explicit instructions, and an example:


complete_and_print(
    "Give me the zip code for Menlo Park in JSON format with the field 'zip_code'",
    model = LLAMA2_70B_CHAT,
)
# Likely returns the JSON and also "Sure! Here's the JSON..."

complete_and_print(
    """
    You are a robot that only outputs JSON.
    You reply in JSON format with the field 'zip_code'.
    Example question: What is the zip code of the Empire State Building? Example answer: {'zip_code': 10118}
    Now here is my question: What is the zip code of Menlo Park?
    """,
    model = LLAMA2_70B_CHAT,
)
# "{'zip_code': 94025}"



In [ ]:
######## REWRITE STUFF #############
customRewritePrompt_System = ("I ask you to rewrite the given text keeping the same meaning but with your own workds, do not copy paste sequence of words from the source. Replace the First-Person Pronouns in source with Third-person Pronouns. Do not use words like we, We, Our, and our.")

questionRewritePrompt = "Rewrite me the following text {text}"
rewrite_prompt_template = get_prompt(questionRewritePrompt, new_system_prompt=customRewritePrompt_System)
print(f"Current rewrite prompt: {rewrite_prompt_template}")
rewrite_prompt = PromptTemplate(template=rewrite_prompt_template, input_variables=["text"])
rewrite_chain = LLMChain(prompt=rewrite_prompt, llm=llm)



In [ ]:
######### Summarize A TEXT ###########
summarizationPrompt_User_template= "Summarize the following article for me {text} with no more that {max_sentences} sentences. Replace the First-Person Pronouns in source with Third-person Pronouns. Do not use words like we or our."
summarizationPrompt_System_template  = "You are an expert on summarization and expressing key ideas succinctly. You speak as a top computer science professor. Do not copy paste sentences from the source."
summarizationPrompt_template = get_prompt(summarizationPrompt_User_template, new_system_prompt=summarizationPrompt_System_template)
print(f"Current summarization prompt: {summarizationPrompt_template}")
summarizationPrompt = PromptTemplate(template=summarizationPrompt_template, input_variables=["text", "max_sentences"])
summarizationChain = LLMChain(prompt=summarizationPrompt, llm=llm)


In [ ]:
def rewrite_text(text: str) -> str:
    res = rewrite_chain.run({"text":text})
    return res 

def summarize_text(text: str, max_sentences : int, num_runs) -> str:
    for i in range(num_runs):
        text = summarizationChain.run({"text":text, "max_sentences":max_sentences})
    return text 


In [ ]:
text = """
IoT devices are increasingly being implicated in cyber-attacks, raising community concern about the risks they pose to critical infrastructure, corporations, and citizens. In order to reduce this risk, the IETF is pushing IoT vendors to develop formal specifications of the intended purpose of their IoT devices, in the form of a Manufacturer Usage Description (MUD), so that their network behavior in any operating environment can be locked down and verified rigorously. This article aims to assist IoT manufacturers in developing and verifying MUD profiles, while also helping adopters of these devices to ensure they are compatible with their organizational policies and track device network behavior using their MUD profile. Our first contribution is to develop a tool that takes the traffic trace of an arbitrary IoT device as input and automatically generates the MUD profile for it. We contribute our tool as open source, apply it to 28 consumer IoT devices, and highlight insights and challenges encountered in the process. Our second contribution is to apply a formal semantic framework that not only validates a given MUD profile for consistency, but also checks its compatibility with a given organizational policy. We apply our framework to representative organizations and selected devices, to demonstrate how MUD can reduce the effort needed for IoT acceptance testing. Finally, we show how operators can dynamically identify IoT devices using known MUD profiles and monitor their behavioral changes in their network."""
res = rewrite_text(text)
#parse_print_text(res)

In [ ]:
text = """
In this project, the NCCoE demonstrated the ability to ensure that when an IoT device connects to a home or small-business network, MUD can automatically permit the device to send and receive only the traffic it requires to perform its intended function. The NCCoE has produced a practice guide to demonstrate the practicality and effectiveness of using the Internet Engineering Task Force’s Manufacturer Usage Description (MUD) standard to strengthen security for IoT devices on home and small-business networks.
"""

text = summarize_text(text, max_sentences=3, num_runs=1)


In [ ]:
"""
import sys 
sys.path.append("../")
from etl.pdfs import concatenate_content_multiple_pages, extract_pdf

documents = extract_pdf({'singlepdf': "../data/special_smart_home/IoT_Dataset___Functional_Testing_Framework.pdf"})
res = concatenate_content_multiple_pages(documents)
text = summarize_text(res, max_sentences=5, num_runs=1)
"""